In [7]:
#Joseph P. Kolly
#FARS Data Analysis
#05/02/25

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import os

#Load Fars data
def loadData(year, data_path = "./fars_data"):

    accidents = pd.read_csv(f"{data_path}/accident.csv", encoding='cp1252')
    vehicles = pd.read_csv(f"{data_path}/vehicle.csv", encoding='cp1252')
    persons = pd.read_csv(f"{data_path}/person.csv", encoding='cp1252')

    print(f"Loaded data for {year}")
    print(f"Accident records: {len(accidents)}")
    print(f"Vehicle records: {len(vehicles)}")
    print(f"Person records: {len(persons)}")

    return accidents, vehicles, persons

#Clean the data and merge the datasets
def prepData(accidents, vehicles, persons):

    #Remove incomplete and invalid accident records
    accidents = accidents.dropna(subset=['ST_CASE', 'HOUR', 'DAY_WEEK', 'RUR_URB'])
    accidents = accidents[accidents['HOUR'].between(0,23)]
    accidents = accidents[accidents['DAY_WEEK'].between(1,7)]
    accidents = accidents[accidents['RUR_URB'].isin([1,2])]

    valid_cases = set(accidents['ST_CASE'])

    #Filter vehicles and persons to match valid accident cases
    vehicles = vehicles[vehicles['ST_CASE'].isin(valid_cases)]
    persons = persons[persons['ST_CASE'].isin(valid_cases)]

    #Remove inaccurate ages
    persons = persons.dropna(subset=['AGE'])
    persons = persons[persons['AGE'] < 120]
    valid_cases = set(persons['ST_CASE'])

    #Final filtering to make sure each set is consistent
    accidents = accidents[accidents['ST_CASE'].isin(valid_cases)]
    vehicles = vehicles[vehicles['ST_CASE'].isin(valid_cases)]

    merged_set = pd.merge(
        accidents,
        vehicles,
        on=['ST_CASE'],
        how='inner',
        suffixes=('_acc', '_veh')
    )

    merge_cols = ['ST_CASE']

    if 'VEH_NO' in merged_set.columns and 'VEH_NO' in persons.columns:
        merge_cols.append('VEH_NO')

    merged_set = pd.merge(
        merged_set,
        persons,
        on=merge_cols,
        how='inner',
        suffixes=('', '_per')
    )

    print("Cleaned the data and merged the datasets")

    return merged_set

def createVisualizations(data, output_dir="./results"):

    os.makedirs(output_dir, exist_ok=True)

    #Fatalities by time of day
    plt.figure(figsize=(10,6))
    sns.histplot(data['HOUR'], bins=24)
    plt.title('Fatal Crashes by Hour of Day')
    plt.xlabel('Hour')
    plt.ylabel('Number of Crashes')
    plt.savefig(f"{output_dir}/crashes_by_hour.png")
    plt.close()

    #Fatalities by dat of week
    plt.figure(figsize=(10,6))
    day_order = ['DAY_WEEK'[i] for i in range(1,8)]
    crash_counts = data['DAY_WEEK'].value_counts().reindex(day_order)
    sns.barplot(x=crash_counts.index, y=crash_counts.values)
    plt.title('Fatal Crashes by Day of Week (1=Sun-7=Sat )')
    plt.xlabel('Day')
    plt.ylabel('Number of Crashes')
    plt.savefig(f"{output_dir}/crashes_by_day.png")
    plt.close()

    #Urban vs Rural
    if 'RUR_URB' in data.columns:
        area_map = {1: 'Rural', 2: 'Urban', 9: 'Unknown'}
        data['AREA'] = data['RUR_URB'].map(area_map)
        area_counts = data['AREA'].value_counts()
        plt.figure(figsize=(8,6))
        sns.barplot(x=area_counts.index, y=area_counts.values)
        plt.title('Fatal Crashes: Urban vs Rural')
        plt.xlabel('Area Type')
        plt.ylabel('Number of Crashes')
        plt.savefig(f"{output_dir}/urban_vs_rural.png")
        plt.close()

    #Age distribution analysis
    if 'AGE' in data.columns:
        plt.figure(figsize=(12,6))
        sns.histplot(data['AGE'], bins=20)
        plt.title('Age Distribution in Fatal Crashes')
        plt.xlabel('Age')
        plt.ylabel('Count')
        plt.savefig(f"{output_dir}/age_distribution.png")
        plt.close()

        age_bins = [0,16,21,25,35,45,55,65,75,120]
        age_labels = ['0-15', '16-20', '21-24', '25-34', '35-44', '45-54', '55-64', '65-74', '75+']
        data['AGE_GROUP'] = pd.cut(data['AGE'], bins=age_bins, labels=age_labels)
        age_group_counts = data['AGE_GROUP'].value_counts().sort_index()
        
        plt.figure(figsize=(12,6))
        sns.histplot(x=age_group_counts.index, y=age_group_counts.values)
        plt.title('Fatal Crashes by Age Group')
        plt.xlabel('Age Group')
        plt.ylabel('Count')
        plt.savefig(f"{output_dir}/crashes_by_age_group.png")
        plt.close()
    
    return data

#Add new features
def add_features(data):

    #Time of day
    data['TIME_CATEGORY'] = pd.cut(
        data['HOUR'],
        bins=[-1,5,11,17,23],
        labels=['Night', 'Morning', 'Afternoon', 'Evening'],
    )

    #Weekend
    data['IS_WEEKEND'] = data['DAY_WEEK'].isin([1,7]).astype(int)

    #Alcohol involved
    if 'DRUNK_DR' in data.columns:
        data['ALCOHOL_INVOLVED'] = (data['DRUNK_DR'] > 0).astype(int)

    #Age Group
    data['YOUNG_DRIVER'] = (data['AGE'] < 25).astype(int)
    data['TEEN_DRIVER'] = (data['AGE'] < 21).astype(int)
    data['SENIOR_DRIVER'] = (data['AGE'] >= 65).astype(int)
    data['ELDERLY_DRIVER'] = (data['AGE'] > 75).astype(int)
    
    return data

#Age Specific analysis
def analyze_age_factors(data, output_dir="./results"):

    #Time of day patterns by age group heatmap
    time_age_count = pd.crosstab(data['TIME_CATEGORY'], data['AGE_GROUP'])
    time_age_pct = time_age_count.div(time_age_count.sum(axis=0), axis=1) * 100

    plt.figure(figsize=(14,18))
    sns.heatmap(time_age_pct, annot=True, fmt='.1f')
    plt.title('Distribution of Crashes by Time of Day and Age Group %')
    plt.xlabel('Age Group')
    plt.ylabel('Time of Day')
    plt.tight_layout()
    plt.savefig(f"{output_dir}/age_time_category_heatmap.png")
    plt.close()

    #Alcohol involvement by age group
    alc_by_age = data.groupby('AGE_GROUP')['ALCOHOL_INVOLVED'].mean() * 100

    plt.figure(figsize=(12,6))
    sns.barplot(x=alc_by_age.index, y=alc_by_age.values)
    plt.title('Percentage of alcohol Involvement by Age Group')
    plt.xlabel('Age Group')
    plt.ylabel('Alc-Related (%)')
    plt.ylim(0, max(alc_by_age.values) * 1.2)
    plt.tight_layout()
    plt.savefig(f"{output_dir}/alc_by_age.png")
    plt.close()

    #Day of week by age group
    data['DAY_NUM'] = data['DAY_WEEK']

    day_age_counts = pd.crosstab(data['DAY_NUM'], data['AGE_GROUP'])
    day_age_pct = day_age_counts.div(day_age_counts.sum(axis=0), axis=1) * 100
    day_age_pct = day_age_pct.sort_index()

    plt.figure(figsize=(14,8))
    sns.heatmap(day_age_pct, annot=True, fmt='.1f')
    plt.title('Distribution of Crashes by Day of Week and Age Group %')
    plt.xlabel('Age Group')
    plt.ylabel('Day of Week (1=Sunday-7=Saturday)')
    plt.tight_layout()
    plt.savefig(f"{output_dir}/age_day_heatmap.png")
    plt.close()

    #Urban vs Rural

    area_age_counts = pd.crosstab(data['AGE_GROUP'], data['AREA'])
    area_age_pct = area_age_counts.div(area_age_counts.sum(axis=1), axis=0) * 100

    if 'Urban' in area_age_pct.columns:
        urban_pct = area_age_pct['Urban'].sort_index()

        plt.figure(figsize=(12,6))
        sns.barplot(x=urban_pct.index, y=urban_pct.values)
        plt.title('Percentage of Urban Crashes by Age Group')
        plt.xlabel('Age Group')
        plt.ylabel('Urban Crashes %')
        plt.tight_layout()
        plt.savefig(f"{output_dir}/urban_by_age.png")
        plt.close()

    if 'Rural' in area_age_pct.columns:
        rural_pct = area_age_pct['Rural'].sort_index()

        plt.figure(figsize=(12,6))
        sns.barplot(x=rural_pct.index, y=rural_pct.values)
        plt.title('Percentage of Rural Crashes by Age Group')
        plt.xlabel('Age Group')
        plt.ylabel('Rural Crashes %')
        plt.tight_layout()
        plt.savefig(f"{output_dir}/rural_by_age.png")
        plt.close()

    return data

#Build the predeictive model
def build_model(data, target='FATALS'):

    #Select features
    features = [
        'HOUR', 'DAY_WEEK', 'IS_WEEKEND', 'ALCOHOL_INVOLVED', 
         'YOUNG_DRIVER', 'TEEN_DRIVER', 
        'SENIOR_DRIVER', 'ELDERLY_DRIVER'
    ]

    #Prep x and y and split data
    x = pd.get_dummies(data[features], drop_first=True)
    y = data[target]

    x_train, x_test, y_train, y_test = train_test_split(
        x, y, test_size=0.3, random_state=42
    )

    #Train the model
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(x_train, y_train)

    #Evaluate
    y_pred = model.predict(x_test)
    print("\nModel evaluation:")
    print(classification_report(y_test, y_pred))

    #Feature importance
    importance = pd.DataFrame({
        'Feature': x.columns,
        'Importance': model.feature_importances_
    }).sort_values('Importance', ascending=False)

    print("\nTop features contributing to fatalities:")
    print(importance.head(10))

    plt.figure(figsize=(12,8))
    sns.barplot(x='Importance', y='Feature', data=importance.head(10))
    plt.title('Feature Importance in Predicting Fatalities')
    plt.tight_layout()
    plt.savefig(f"./results/feature_importance.png")
    plt.close()

    return model, importance


    
    


In [8]:
#Main Function

#set the year
year = 2019

os.makedirs("./results", exist_ok=True)

accidents, vehicles, persons = loadData(year)

preped_data = prepData(accidents, vehicles, persons)

preped_data = createVisualizations(preped_data)

preped_data = add_features(preped_data)

preped_data = analyze_age_factors(preped_data)

model, importance = build_model(preped_data)

C:\Users\Joey\AppData\Local\Temp\ipykernel_14980\3603116500.py:17: DtypeWarning: Columns (40,42) have mixed types. Specify dtype option on import or set low_memory=False.
  accidents = pd.read_csv(f"{data_path}/accident.csv", encoding='cp1252')
C:\Users\Joey\AppData\Local\Temp\ipykernel_14980\3603116500.py:18: DtypeWarning: Columns (56,58,101,193,195) have mixed types. Specify dtype option on import or set low_memory=False.
  vehicles = pd.read_csv(f"{data_path}/vehicle.csv", encoding='cp1252')
C:\Users\Joey\AppData\Local\Temp\ipykernel_14980\3603116500.py:19: DtypeWarning: Columns (15,106,108) have mixed types. Specify dtype option on import or set low_memory=False.
  persons = pd.read_csv(f"{data_path}/person.csv", encoding='cp1252')


Loaded data for 2019
Accident records: 33487
Vehicle records: 51623
Person records: 82843
Cleaned the data and merged the datasets


C:\Users\Joey\AppData\Local\Temp\ipykernel_14980\3603116500.py:181: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  alc_by_age = data.groupby('AGE_GROUP')['ALCOHOL_INVOLVED'].mean() * 100



Model evaluation:
              precision    recall  f1-score   support

           1       0.88      1.00      0.93     18995
           2       0.40      0.01      0.02      2160
           3       0.00      0.00      0.00       382
           4       0.83      0.04      0.07       141
           5       0.00      0.00      0.00        19
           6       0.00      0.00      0.00         7
           7       0.00      0.00      0.00        17
           8       0.00      0.00      0.00         3

    accuracy                           0.87     21724
   macro avg       0.26      0.13      0.13     21724
weighted avg       0.81      0.87      0.82     21724


Top features contributing to fatalities:
            Feature  Importance
0              HOUR    0.702711
1          DAY_WEEK    0.168852
3  ALCOHOL_INVOLVED    0.039108
5       TEEN_DRIVER    0.028606
4      YOUNG_DRIVER    0.021375
6     SENIOR_DRIVER    0.014650
2        IS_WEEKEND    0.013510
7    ELDERLY_DRIVER    0.011188


c:\Users\Joey\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Joey\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Joey\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif